# COVID-19 Analysis and Prediction: A Modified SEIR infectious Model

## Table of Contents
<ul>
    <li><a href="#introduction">Introduction</a></li>
    <li><a href="#data_wrangling">Data Wrangling</a></li>
    <li><a href="#eda">Exploratory Data Analysis</a></li>
    <li><a href="#conclusion">Conclusions</a></li>
</ul>

<a id="introduction"></a>
## Introduction

The pandemics of COVID-19 in 2020 changed the life of everyone in the world. It is caused by a coronavirus called SARS-CoV-2. Different from most of other infectious disease, people who are infected with COVID-19 but do not exhibit symptoms can also spread the virus to others. Older adults and people who have severe underlying medical conditions like heart or lung disease or diabetes seem to be at higher risk for developing more serious complications from COVID-19 illness. As COVID-19 continues to spread across the country and the world, we should remain calm and take actions to slow down its spreading.

Here, we modified the SEIR model to accounts for the different infection feature of COVID-19 and the "lockdown" rules applied to most places. We studied three of the most serious countries: US, Italy, and France. This model fits the infection ratio and recover ratio very well.

I am keep working on this notebook and learning from the community. If you have any comments or questions, I'm happy to hear and answer.


<a id="data_wrangling"></a>
## Data wrangling
### Import packages

In [ ]:
# Algebra
import numpy as np

# Dataframe
import pandas as pd

# Missing Analysis
import missingno as msno

# Modelling
from scipy import integrate
from scipy import optimize

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Map
import folium

# Datetime
from datetime import datetime

### Import datasets

Date source:
COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University ([GitHub link](https://github.com/CSSEGISandData/COVID-19))

In [ ]:
# Global cases
global_confirm_csv = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
global_death_csv = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
global_recover_csv = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
# US cases
us_confirm_csv = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
us_death_csv = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
us_state_loc_csv = 'https://gist.githubusercontent.com/mbostock/9535021/raw/eaed7e5632735a6609f02d0ba0e55c031e14200d/us-state-capitals.csv'

In [ ]:
Confirmed = pd.read_csv(global_confirm_csv)
Death = pd.read_csv(global_death_csv) 
Recovered = pd.read_csv(global_recover_csv)
Confirmed_us = pd.read_csv(us_confirm_csv)
Death_us = pd.read_csv(us_death_csv)
us_state_location = pd.read_csv(us_state_loc_csv)

### Data cleaning

Settings to show the full dataset when print()

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

#### Global cases

remove unnecessary columns

In [ ]:
# Global cases
Confirmed.drop(axis=1, inplace=True, columns=['Province/State', 'Lat', 'Long'])
Death.drop(axis=1, inplace=True, columns=['Province/State', 'Lat', 'Long'])
Recovered.drop(axis=1, inplace=True, columns=['Province/State', 'Lat', 'Long'])

In [ ]:
display(Confirmed)

In [ ]:
# function to extract the data of different cases in different country
def extract_national_data(df, country, case):
    df_extract = df[df['Country/Region'] == country].sum(axis=0)
    df_extract = df_extract.T.reset_index()
    df_extract = df_extract.iloc[1:]
    df_extract.columns = ['date', case]
    df_extract.date = pd.to_datetime(df_extract.date)
    return df_extract

In [ ]:
# function to join three types of cases in one country
def join_national_data(confirmed, death, recovered):
    total_case = confirmed.join(death.set_index('date'), on='date').join(recovered.set_index('date'), on='date')
    total_case.reset_index(inplace=True, drop=True)
    return total_case

### US

In [ ]:
us_confirmed = extract_national_data(Confirmed, 'US', 'positive')
us_death = extract_national_data(Death, 'US', 'death')
us_recovered = extract_national_data(Recovered, 'US', 'recovered')

In [ ]:
us_status = join_national_data(us_confirmed, us_death, us_recovered)

In [ ]:
us_test = us_status[:100]

In [ ]:
us_status.head()

### Italy

In [ ]:
itl_confirmed = extract_national_data(Confirmed, 'Italy', 'positive')
itl_death = extract_national_data(Death, 'Italy', 'death')
itl_recovered = extract_national_data(Recovered, 'Italy', 'recovered')

In [ ]:
itl_status = join_national_data(itl_confirmed, itl_death, itl_recovered)

In [ ]:
itl_status.head()

### France

In [ ]:
frc_confirmed = extract_national_data(Confirmed, 'France', 'positive')
frc_death = extract_national_data(Death, 'France', 'death')
frc_recovered = extract_national_data(Recovered, 'France', 'recovered')

In [ ]:
frc_status = join_national_data(frc_confirmed, frc_death, frc_recovered)

In [ ]:
display(frc_status.head())

<a id="eda"></a>
## Exploratory Data Analysis
## The modified SEIRS Model

Inspired by Kaike Wesley Reis's work on SEIR model in Kaggle community, I generated a modified SEIRS model for COVID-19.
More information about SEIRS model could be found [here.](https://www.idmod.org/docs/hiv/model-seir.html#:~:text=The%20SEIR%20model%20assumes%20people,return%20to%20a%20susceptible%20state)

In a closed population without births or deaths, the SEIRS model is:

<a href="https://www.codecogs.com/eqnedit.php?latex=\frac{\mathrm{d}S&space;}{\mathrm{d}&space;t}=&space;-\beta&space;SI&plus;\xi&space;R" target="_blank"><img src="https://latex.codecogs.com/gif.latex?\frac{\mathrm{d}S&space;}{\mathrm{d}&space;t}=&space;-\beta&space;SI&plus;\xi&space;R" title="\frac{\mathrm{d}S }{\mathrm{d} t}= -\beta SI+\xi R" /></a>

<a href="https://www.codecogs.com/eqnedit.php?latex=\frac{\mathrm{d}E&space;}{\mathrm{d}&space;t}=\frac{\beta&space;SI}{N}-\sigma&space;E" target="_blank"><img src="https://latex.codecogs.com/gif.latex?\frac{\mathrm{d}E&space;}{\mathrm{d}&space;t}=\beta&space;SI-\sigma&space;E" title="\frac{\mathrm{d}E }{\mathrm{d} t}=\beta SI-\sigma E" /></a>

<a href="https://www.codecogs.com/eqnedit.php?latex=\frac{\mathrm{d}I&space;}{\mathrm{d}&space;t}=\sigma&space;E-\gamma&space;I" target="_blank"><img src="https://latex.codecogs.com/gif.latex?\frac{\mathrm{d}I&space;}{\mathrm{d}&space;t}=\sigma&space;E-\gamma&space;I" title="\frac{\mathrm{d}I }{\mathrm{d} t}=\sigma E-\gamma I" /></a>

<a href="https://www.codecogs.com/eqnedit.php?latex=\frac{\mathrm{d}R&space;}{\mathrm{d}&space;t}=&space;\gamma&space;I&space;-&space;\xi&space;R" target="_blank"><img src="https://latex.codecogs.com/gif.latex?\frac{\mathrm{d}R&space;}{\mathrm{d}&space;t}=&space;\gamma&space;I&space;-&space;\xi&space;R" title="\frac{\mathrm{d}R }{\mathrm{d} t}= \gamma I - \xi R" /></a>

Where S, E, I, R is the proportion of suceptible, exposed, infectious and recovered population.

In the case of COVID-19, the viral carriers ("exposed population) do not exhibit symptoms, yet are infectious. So, the SEIR model is modified as below:

<a href="https://www.codecogs.com/eqnedit.php?latex=\frac{\mathrm{d}S&space;}{\mathrm{d}&space;t}=&space;-\frac{\beta&space;S(I&plus;E)}{N}&plus;\xi&space;R" target="_blank"><img src="https://latex.codecogs.com/gif.latex?\frac{\mathrm{d}S&space;}{\mathrm{d}&space;t}=&space;-\beta&space;S(I&plus;E)&plus;\xi&space;R" title="\frac{\mathrm{d}S }{\mathrm{d} t}= -\frac{\beta S(I+E)}{N}+\xi R" /></a>

<a href="https://www.codecogs.com/eqnedit.php?latex=\frac{\mathrm{d}E&space;}{\mathrm{d}&space;t}=\frac{\beta&space;S(I&plus;E)}{N}-\sigma&space;E" target="_blank"><img src="https://latex.codecogs.com/gif.latex?\frac{\mathrm{d}E&space;}{\mathrm{d}&space;t}=\beta&space;S(I&plus;E)-\sigma&space;E" title="\frac{\mathrm{d}E }{\mathrm{d} t}=\frac{\beta S(I+E)}{N}-\sigma E" /></a>

<a href="https://www.codecogs.com/eqnedit.php?latex=\frac{\mathrm{d}I&space;}{\mathrm{d}&space;t}=\sigma&space;E-\gamma&space;I" target="_blank"><img src="https://latex.codecogs.com/gif.latex?\frac{\mathrm{d}I&space;}{\mathrm{d}&space;t}=\sigma&space;E-\gamma&space;I" title="\frac{\mathrm{d}I }{\mathrm{d} t}=\sigma E-\gamma I" /></a>


<a href="https://www.codecogs.com/eqnedit.php?latex=\frac{\mathrm{d}R&space;}{\mathrm{d}&space;t}=&space;\gamma&space;I&space;-&space;\xi&space;R" target="_blank"><img src="https://latex.codecogs.com/gif.latex?\frac{\mathrm{d}R&space;}{\mathrm{d}&space;t}=&space;\gamma&space;I&space;-&space;\xi&space;R" title="\frac{\mathrm{d}R }{\mathrm{d} t}= \gamma I - \xi R" /></a>

Where S, E, I, R is the proportion of suceptible, exposed, infectious, and recovered population.

Here, we included the "exposed population" in the first and second formula, because it participates in the viral transmission.

Accordingly, we modified the functions as below:

In [ ]:
# Function
def seir_model_ode(y, t, params): 
    '''
    Arguments:
    - y: dependent variables
    - t: independent variable (time)
    - params: Model parameters
    '''
    # Parameters to find
    infection_rate = params[0]
    recovery_rate = params[1]
    exposed_rate = params[2]
    reinfection_rate = params[3]
    
    # Y variables
    s = y[0]
    e = y[1]
    i = y[2]
    r = y[3]
    
    # SIR ODE System 
    dsdt = -exposed_rate*s*(i+e) + reinfection_rate*r
    dedt = (exposed_rate*s*(i+e)) - (infection_rate*e)
    didt = (infection_rate*e) - (recovery_rate*i)
    drdt = recovery_rate*i - reinfection_rate*r
    
    # Return our system
    return (dsdt, dedt, didt, drdt)

In [ ]:
# FUNCTION - Calculate SEIR Model in t (time as days) based on given parameters
def calculate_seir_model(params, t, initial_condition):
    # Create an alias to our seir ode model to pass params to try
    seir_ode = lambda y,t:seir_model_ode(y,t, params)
    
    # Calculate ode solution, return values to each
    ode_result = integrate.odeint(func=seir_ode, y0=initial_condition, t=t)
    
    # Return results
    return ode_result

In [ ]:
# FUNCTION - Auxiliar function to find the best parameters
def fit_seir_model(params_to_fit, t, initial_condition, i_r_true):
    # Calculate ODE solution for possible parameter, return values to each dependent variable:
    # (s, e, i and r)
    fit_result = calculate_seir_model(params_to_fit, t, initial_condition)
    
    # Calculate residual value between predicted VS true
    ## Note: ode_result[0] is S result
    residual_i = i_r_true[0] - fit_result[:,2]
    residual_r = i_r_true[1] - fit_result[:,3]
    
    # Create a np.array of all residual values for both (i) and (r)
    residual = np.concatenate((residual_i, residual_r))
    
    # Return results
    return residual

In [ ]:
def countryLockdown(N, status,lockdownDay,countryName):
    
    """inputs:
    N: population in the country
    status: dataframe of the country
    lockdownDay: number of days since begining of data to the date of lockdown
    """
    
    # Define Initial Condition before lockdown
    I_start = status.loc[0, 'positive']/N
    E_start = (status.loc[14, 'positive'] - status.loc[0, 'positive'])/N
    S_start = 1 - E_start - I_start
    R_start = status.loc[0, 'recovered']/N
    
    # Set this values as a tuple of initial condition
    ic = (S_start, E_start, I_start, R_start)
    
    # Create a tuple with the true values in fraction for Infected/Recovered cases (necessary for error measurement)
    beforelockdown=status.loc[0:lockdownDay]
    afterlockdown=status.loc[lockdownDay+1:]
    i_r_true_bf = (list(beforelockdown['positive']/N), list(beforelockdown['recovered']/N))
    i_r_true_af = (list(afterlockdown['positive']/N), list(afterlockdown['recovered']/N))

    # Define a time array measure in days
    time_opt_bf = range(0, lockdownDay+1)
    time_opt_af = range(0, len(afterlockdown))
    time_opt =range(0,len(status))
    
    
    # define initial condition after lockdown
    E_start_day = min(len(status['date']),len(beforelockdown)+14)
    
    E_start_af = (status.loc[E_start_day, 'positive'] - status.loc[len(beforelockdown), 'positive'])/N
    I_start_af = status.loc[len(beforelockdown), 'positive']/N
    S_start_af = 1 - E_start_af - I_start_af
    R_start_af = status.loc[len(beforelockdown), 'recovered']/N
   
    # Set this values as a tuple
    ic_af = (S_start_af, E_start_af, I_start_af, R_start_af)
    
    # Define a start guess for our parameters [infection_rate, recovered rate]
    params_start_guess = [0.01, 0.001, 0.01, 0.001]
    optimal_params, success = optimize.leastsq(fit_seir_model,
                                          x0=params_start_guess,
                                          args=(time_opt_bf, ic, i_r_true_bf),
                                          ftol=1.49012e-22)
    optimal_params_af, success = optimize.leastsq(fit_seir_model,
                                          x0=params_start_guess,
                                          args=(time_opt_af, ic_af, i_r_true_af),
                                          ftol=1.49012e-22)
    
    print('## '+countryName+' before lockdown')
    print('Optimized infection rate: ', optimal_params[0])
    print('Optimized recovered rate: ', optimal_params[1])
    print('Optimized exposed rate: ', optimal_params[2])
    print('Optimized reinfection rate: ', optimal_params[3])
    print('\n')
    print('## '+countryName+' after lockdown')
    print('Optimize infection rate: ', optimal_params_af[0])
    print('Optimize recovered rate: ', optimal_params_af[1])
    print('Optimize exposed rate: ', optimal_params_af[2])
    print('Optimized reinfection rate: ', optimal_params_af[3])
            
            
    # Fit test
    ## Get the optimal parameters
    ir = optimal_params[0]
    rr = optimal_params[1]
    er = optimal_params[2]
    rir = optimal_params[3]
    ir_af = optimal_params_af[0]
    rr_af = optimal_params_af[1]
    er_af = optimal_params_af[2]
    rir_af = optimal_params_af[3]
    
    ## Calculate a curve based on those parameters
    fit_result_bf = calculate_seir_model((ir, rr, er, rir), time_opt_bf, ic)
    fit_result_af = calculate_seir_model((ir_af, rr_af, er_af, rir_af), time_opt_af, ic_af)

    ## Define plot object
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True, sharex=True, figsize=[18, 5])
    fig.text(0.5, 0.04, 'Number of days after Jan-22nd 2020', ha='center', fontsize=12)
    ## Plot real and predicted infection
    ax1.set_title('Infected cases in '+ countryName,fontsize=20)
    ax1.plot(time_opt, i_r_true_bf[0]+i_r_true_af[0], 'ro', markersize = 0.3)
    ax1.plot(time_opt, np.hstack((fit_result_bf[:,2],fit_result_af[:,2])), 'co', markersize=0.3)
    ax1.legend(['Actual infection', 'Predicted infection'],loc=2, fontsize=12)
    ax1.set_ylabel('Proportion of population', fontsize=12)
    
    ## Plot real and predicted recover
    ax2.set_title('Recovered cases in '+countryName,fontsize=20)      
    ax2.plot(time_opt, i_r_true_bf[1]+i_r_true_af[1], 'ro', markersize = .3)
    ax2.plot(time_opt, np.hstack((fit_result_bf[:,3],fit_result_af[:,3])), 'go', markersize=0.3)
    ax2.legend(['Real recover', 'Predicted recover'],loc=2, fontsize=12)
    

    
    # Prediction
    ## Get prediction full period time in datetime object and the convert to string
    datetime_pred = pd.date_range(start="2020-02-01",end="2021-01-01", freq='D')
    pred_time = [x.strftime("%Y-%m-%d") for x in datetime_pred]
    pred_range = range(0, len(pred_time))
    pred_result = calculate_seir_model((ir_af, rr_af, er_af, rir_af), pred_range, ic_af)
    time_axis = [pred_time[i] for i in[0, 29, 60, 90, 121, 151, 182, 213, 243, 274, 304, 335]]
    time_labels = ['Feb.', 'Mar.', 'Apr.', 'May', 'June', 'July', 'Aug.', 'Sept.', 'Oct.', 'Nov.', 'Dec.', 'Jan.']
    ## Plot SEIDR
    fig, ax = plt.subplots(figsize=[12,10])
    ax.plot(pred_time, pred_result[:,0],color='blue') #susceptible
    ax.plot(pred_time, pred_result[:,1],color='red') #exposed
    ax.plot(pred_time, pred_result[:,2],color='cyan') #infected
    ax.plot(pred_time, pred_result[:,3], color = 'black') #recovered
    ax.legend(loc=1, labels=['Susceptible', 'Exposed', 'Infected', 'Recovered'], fontsize=8)
    ax.set_title('SEIR predictions', fontsize=20)
    ax.set_xlabel('Month', fontsize=12)
    ax.set_ylabel('Proportion of population', fontsize=12)
    plt.xticks(time_axis, time_labels, rotation='vertical');

In [ ]:
def countrynoLockdown(N, status,countryName):
    
    """inputs:
    N: population in the country
    status: dataframe of the country
    lockdownDay: number of days since begining of data to the date of lockdown
    """
    
    # Define Initial Condition before lockdown
    I_start = status.loc[0, 'positive']/N
    E_start = (status.loc[14, 'positive'] - status.loc[0, 'positive'])/N
    S_start = 1 - E_start - I_start
    R_start = status.loc[0, 'recovered']/N
    
    # Set this values as a tuple of initial condition
    ic = (S_start, E_start, I_start, R_start)
    
    # Create a tuple with the true values in fraction for Infected/Recovered cases (necessary for error measurement)
   
    i_r_true = (list(status['positive']/N), list(status['recovered']/N))
    

    # Define a time array measure in days
    time_opt =range(0,len(status))
    
    
    # Define a start guess for our parameters [infection_rate, recovered rate]
    params_start_guess = [0.01, 0.001, 0.01, 0.001]
    optimal_params, success = optimize.leastsq(fit_seir_model,
                                          x0=params_start_guess,
                                          args=(time_opt, ic, i_r_true),
                                          ftol=1.49012e-22)
    
    
    print('## '+countryName+':')
    print('Optimized infection rate: ', optimal_params[0])
    print('Optimized recovered rate: ', optimal_params[1])
    print('Optimized exposed rate: ', optimal_params[2])
    print('Optimized reinfection rate: ', optimal_params[3])

            
    # Fit test
    ## Get the optimal parameters
    ir = optimal_params[0]
    rr = optimal_params[1]
    er = optimal_params[2]
    rir = optimal_params[3]
    
    ## Calculate a curve based on those parameters
    fit_result = calculate_seir_model((ir, rr, er, rir), time_opt, ic)
    ## Define plot object
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True, sharex=True, figsize=[18, 5])
    fig.text(0.5, 0.04, 'Number of days after Jan-22nd 2020', ha='center', fontsize=12)
    ## Plot real and predicted infection
    ax1.set_title('Infected cases in '+ countryName,fontsize=20)
    ax1.plot(time_opt, i_r_true[0], 'ro', markersize = 0.3)
    ax1.plot(time_opt, fit_result[:,2], 'co', markersize=0.3)
    ax1.legend(['Actual infection', 'Predicted infection'],loc=2, fontsize=12)
    ax1.set_ylabel('Proportion of population', fontsize=12)
    
    ## Plot real and predicted recover
    ax2.set_title('Recovered cases in '+countryName,fontsize=20)      
    ax2.plot(time_opt, i_r_true[1], 'ro', markersize = .3)
    ax2.plot(time_opt, fit_result[:,3], 'go', markersize=0.3)
    ax2.legend(['Real recover', 'Predicted recover'],loc=2, fontsize=12)
    

    
    # Prediction
    ## Get prediction full period time in datetime object and the convert to string
    datetime_pred = pd.date_range(start="2020-02-01",end="2021-01-01", freq='D')
    pred_time = [x.strftime("%Y-%m-%d") for x in datetime_pred]
    pred_range = range(0, len(pred_time))
    pred_result = calculate_seir_model((ir, rr, er, rir), pred_range, ic)
    time_axis = [pred_time[i] for i in[0, 29, 60, 90, 121, 151, 182, 213, 243, 274, 304, 335]]
    time_labels = ['Feb.', 'Mar.', 'Apr.', 'May', 'June', 'July', 'Aug.', 'Sept.', 'Oct.', 'Nov.', 'Dec.', 'Jan.']
    ## Plot SEIDR
    fig, ax = plt.subplots(figsize=[12,10])
    ax.plot(pred_time, pred_result[:,0],color='blue') #susceptible
    ax.plot(pred_time, pred_result[:,1],color='red') #exposed
    ax.plot(pred_time, pred_result[:,2],color='cyan') #infected
    ax.plot(pred_time, pred_result[:,3], color = 'black') #recovered
    ax.legend(loc=1, labels=['Susceptible', 'Exposed', 'Infected', 'Recovered'], fontsize=8)
    ax.set_title('SEIR predictions', fontsize=20)
    ax.set_xlabel('Month', fontsize=12)
    ax.set_ylabel('Proportion of population', fontsize=12)
    plt.xticks(time_axis, time_labels, rotation='vertical');

# Case study
# United States

In [ ]:
countryLockdown(328*1e6,us_status,58,'US');

# Italy

In [ ]:
countryLockdown(60.36*1e6, itl_status,47,'Italy')

# France

In [ ]:
countryLockdown(66.99*1e6,frc_status,55,'France');

<a id="conclusion"></a>
## Conclusion
This model fits well with the current data in US, Italy and France. 
As COVID-19 continues to spread across the country and the world, we should remain calm and take actions to slow down its spreading.
